# Run pyCIAM in a Diaz 2016 configuration, using Diaz 2016 Inputs

## Setup

In [2]:
import sys

sys.path.append("../")

In [3]:
from pathlib import Path

import distributed as dd
import pandas as pd
from pyCIAM.run import execute_pyciam
from shared import (
    AUTHOR,
    CONTACT,
    DIR_SCRATCH,
    HISTORY,
    PATH_DIAZ_INPUTS_INT,
    PATH_DIAZ_RES,
    PATH_PARAMS_DIAZ,
    STORAGE_OPTIONS,
    save,
)

In [4]:
# number of seg to run at once
SEG_CHUNKSIZE = 4050

# parameter object
PARAMS = pd.read_json(PATH_PARAMS_DIAZ)["values"]

# attrs
DESCRIPTION = (
    "Projected coastal damages from the Coastal Impact and Adaptation Model (CIAM, "
    "Diaz 2016), using original Diaz 2016 inputs and assumptions."
)

HISTORY = (
    """Replication of Diaz 2016.

"""
    + HISTORY
)

In [5]:
# Small enough to run locally so we just use a local cluster by default here
client = dd.Client(threads_per_worker=1, n_workers=3)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/bolliger32/proxy/8787/status,
Dashboard: /user/bolliger32/proxy/8787/status,Workers: 3
Total threads: 3,Total memory: 45.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:43269,Workers: 3
Dashboard: /user/bolliger32/proxy/8787/status,Total threads: 3
Started: Just now,Total memory: 45.00 GiB
Comm: tcp://127.0.0.1:43653,Total threads: 1
Dashboard: /user/bolliger32/proxy/34165/status,Memory: 15.00 GiB
Nanny: tcp://127.0.0.1:32959,


In [8]:
out = execute_pyciam(
    PATH_PARAMS_DIAZ,
    PATH_DIAZ_INPUTS_INT,
    [None],
    [None],
    DIR_SCRATCH / "refA-diaz.zarr",
    seg_var="seg",
    quantiles=None,
    extra_attrs={
        "author": AUTHOR,
        "history": HISTORY,
        "contact": CONTACT,
        "description": DESCRIPTION,
    },
    refA_seg_chunksize=SEG_CHUNKSIZE,
    pyciam_seg_chunksize=SEG_CHUNKSIZE,
    diaz_inputs=True,
    diaz_config=True,
    dask_client_func=lambda: client,
    storage_options=STORAGE_OPTIONS,
)

In [14]:
for k, v in out.coords.items():
    if v.dtype == object:
        out[k] = v.astype("unicode")
save(out, PATH_DIAZ_RES, mode="w")